In [ ]:
import torch
import os

print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name(s):", [torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())])
# Get Slurm-allocated CPU count or fallback
n_jobs = int(os.environ.get("SLURM_CPUS_PER_TASK", os.cpu_count()))
print("Number of jobs:", n_jobs)
n_human = int(n_jobs/4)

In [ ]:
params = {
    'model': 'lorenz',
    'case': 'synthetic_lorenz',
    'input_dim': 80,
    'latent_dim': 3,
    'poly_order': 2,
    'include_sine': False,
    'fix_coefs': False,
    'svd_dim': 40,
    'delay_embedded': True,
    'scale': True,
    'coefficient_initialization': 'constant',
    "coefficient_initial_value": 1e-2,
    'widths_ratios': [1.0, 0.6],

    # Training
    'max_epochs': 500,
    'patience': 20,
    'batch_size': 2**9,
    'learning_rate': 1e-3,
    "lr_decay": 0.999,

    # Loss Weights
    'loss_weight_rec': 0.3,
    'loss_weight_sindy_z': 0.001,
    'loss_weight_sindy_x': 0.001,
    'loss_weight_sindy_regularization': 1e-5,
    'loss_weight_integral': 0.1,
    'loss_weight_x0': 0.01,
    'loss_weight_layer_l2': 0.0,
    'loss_weight_layer_l1': 0.0,

    # SINDy
    'coefficient_threshold': 1e-5,
    'threshold_frequency': 5,
    'print_frequency': 10,
    'sindy_pert': 0.0,
    'ode_net': False,
    'ode_net_widths': [1.5, 2.0],
    'exact_features': True,
    'use_bias': True,

    # Misc
    'train_ratio': 0.8,
    'data_path': './trained_models',
    'save_checkpoints': False,
    'save_freq': 5,
    'learning_rate_sched': False,
    'use_sindycall': False,
    'sparse_weighting': None,
    'system_coefficients': None
}

In [ ]:
import torch
import time
from TrainingCode.TrainingCode import TrainModel  # adjust import path if needed
from ParamOptCode.DefaultParams import params
from AdditionalFunctionsCode.SolverFunctions import SynthData
from DataGenerationCode.DataGeneration import LorenzSystem

# === SETUP ===
params["max_epochs"] = 500
params["input_dim"] = params["svd_dim"]  # ensure consistent dimensions
params["batch_size"] = 2**9  # larger batch to leverage GPU better
params["dt"] = 0.01 

# === Generate synthetic data ===
lorenz_system = LorenzSystem()
synth_data = SynthData(input_dim=params["input_dim"])
z0 = torch.tensor([1.0, 1.0, 1.0])
synth_data.run_sim(tend=32, dt=0.01, z0=z0.numpy(), apply_svd=True, svd_dim=params["svd_dim"])

# === Choose device ===
# Change to 'cpu' to benchmark that version
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")  # Uncomment to test CPU explicitly

# === Train ===
trainer = TrainModel(synth_data, params, device=device)

start = time.time()
trainer.fit()
end = time.time()

device_type = "GPU" if device.type == "cuda" else "CPU"
print(f"\n=== Training on {device_type} ({device}) completed in {(end - start)/60:.2f} minutes ===")

In [ ]:
print(params["svd_dim"])

In [ ]:
# Delete old database
import os
import torch
os.remove("optuna_study.db")  # or your full path

Perform the Hyperparameter Optimization

In [ ]:
from ParamOptCode.ParameterOptimizer import HyperparameterOptimizer
from ParamOptCode.DefaultParams import params 
import time
#os.remove('C:/Users/mehme/OneDrive/Desktop/All/Code/LorenzReclaim/optuna_study.db')
start_time = time.time()
device = torch.device("cpu")
device_type = "GPU" if device.type == "cuda" else "CPU"
params["input_dim"] = params["svd_dim"]
optimizer = HyperparameterOptimizer(n_trials=2, n_jobs=1)
best = optimizer.run()
end_time = time.time()
print(f"Best hyperparameters: {best}")
print(f"Hyperparameter optimization completed in {(end_time - start_time)/60:.2f} minutes")